In [2]:
import pandas as pd
import csv
from loader import CreateMovieBaskets, ReadMovies
from pair_finder import *
import time

In [3]:
ratings_path = "DATA\\ratings_100users.csv"
movies_path =  "DATA\\movies.csv"
MinScore = 4.5 # CHANGE THIS TO USER INPUT

UserBaskets,movies_list = CreateMovieBaskets(ratings_path,MinScore, limit=100)
movies_df = ReadMovies(movies_path)
movies_df = movies_df[movies_df['movieId'].isin(movies_list)].reset_index(drop=True)



In [15]:
def calculate_frequencies(object_list, N):
    vector_dict = {}

    for b_i in range(len(object_list)):
        for c in object_list[b_i]: # c -> combo
            if c not in vector_dict.keys():
                vector_dict[c] = {"frequency": 1, "baskets" : [b_i]}
            else:
                vector_dict[c]['frequency'] += 1
                vector_dict[c]['baskets'].append(b_i)
    for i in vector_dict.keys():
        vector_dict[i]['frequency'] = vector_dict[i]['frequency'] / N
    
    return vector_dict


def frequency_filter(min_frequency, input_dict):
    
    for i in list(input_dict):
        if input_dict[i]['frequency'] < min_frequency:
            del input_dict[i]
    
    return input_dict


def get_combos(ub, L, k):
    combo_list = []

    for i_b, basket in enumerate(ub): # Iterate the baskets
        print("k:", k,"User: ", i_b + 1, "/ 100")
        combo_list.append([])
        if len(basket) >= k: # Baskets with size less than k can't have k-combo

            for i in basket: # Iterate the objects in the basket

                if i in list(L[1]): # If the object not in L1, it isn't frequent
                    
                    for j in list(L[k-1]): # Iterate the L_k-1 dict which holds the frequenct k-1 combos
                        previous_combo_elements = j.split(",")
                        str_i = str(i)
                        if i_b in L[k-1][j]['baskets'] and str_i not in previous_combo_elements:
                            combo_list[-1].append(create_new_combo(str_i,previous_combo_elements))
                        
            combo_list[-1] = list(dict.fromkeys(combo_list[-1]))
            
    return combo_list
                        

def create_new_combo(new_object, old_combo):
    #combo_objects = old_combo.split(",")
    old_combo.append(new_object)
    final_combo = ""
    sorted_combo = sorted(old_combo, reverse=True)
    for i in sorted_combo:
        final_combo += i + ","

    return final_combo[:-1]

In [16]:
def apriori(itemBaskets, min_frequency, max_length):
    L = {}
    C1 = calculate_frequencies(itemBaskets ,len(itemBaskets))
    L[1] = frequency_filter(min_frequency,C1)
    pairs = []
    for i in itemBaskets:
        pairs.append(get_pairs(i))
    C2 = calculate_frequencies(pairs, len(itemBaskets))
    L[2] = frequency_filter(min_frequency, C2)
    
    for i in range(3,max_length + 1):
        tmp_combos = get_combos(itemBaskets, L, i)
        Ci = calculate_frequencies(tmp_combos, len(itemBaskets))
        L[i] = frequency_filter(min_frequency, Ci)
        if len(L[i].keys()) == 0:
            break

    return L    
    



In [19]:
L = apriori(UserBaskets, 0.1, 5)

k: 3 User:  1 / 100
k: 3 User:  2 / 100
k: 3 User:  3 / 100
k: 3 User:  4 / 100
k: 3 User:  5 / 100
k: 3 User:  6 / 100
k: 3 User:  7 / 100
k: 3 User:  8 / 100
k: 3 User:  9 / 100
k: 3 User:  10 / 100
k: 3 User:  11 / 100
k: 3 User:  12 / 100
k: 3 User:  13 / 100
k: 3 User:  14 / 100
k: 3 User:  15 / 100
k: 3 User:  16 / 100
k: 3 User:  17 / 100
k: 3 User:  18 / 100
k: 3 User:  19 / 100
k: 3 User:  20 / 100
k: 3 User:  21 / 100
k: 3 User:  22 / 100
k: 3 User:  23 / 100
k: 3 User:  24 / 100
k: 3 User:  25 / 100
k: 3 User:  26 / 100
k: 3 User:  27 / 100
k: 3 User:  28 / 100
k: 3 User:  29 / 100
k: 3 User:  30 / 100
k: 3 User:  31 / 100
k: 3 User:  32 / 100
k: 3 User:  33 / 100
k: 3 User:  34 / 100
k: 3 User:  35 / 100
k: 3 User:  36 / 100
k: 3 User:  37 / 100
k: 3 User:  38 / 100
k: 3 User:  39 / 100
k: 3 User:  40 / 100
k: 3 User:  41 / 100
k: 3 User:  42 / 100
k: 3 User:  43 / 100
k: 3 User:  44 / 100
k: 3 User:  45 / 100
k: 3 User:  46 / 100
k: 3 User:  47 / 100
k: 3 User:  48 / 100
k

In [20]:
count = 0
for i in L[2].keys():
    print(i, " : ", L[2][i])
    count += 1
count

260,50  :  {'frequency': 0.13, 'baskets': [0, 6, 16, 31, 38, 41, 44, 56, 62, 68, 71, 90, 95]}
527,50  :  {'frequency': 0.1, 'baskets': [0, 7, 16, 17, 32, 37, 56, 57, 71, 95]}
1196,260  :  {'frequency': 0.19, 'baskets': [0, 3, 14, 16, 20, 29, 38, 44, 50, 61, 62, 67, 68, 71, 74, 76, 78, 90, 94]}
1198,260  :  {'frequency': 0.15, 'baskets': [0, 16, 24, 29, 31, 56, 61, 62, 67, 71, 74, 76, 90, 94, 95]}
1198,1196  :  {'frequency': 0.11, 'baskets': [0, 16, 29, 61, 62, 67, 71, 74, 76, 90, 94]}
1210,50  :  {'frequency': 0.11, 'baskets': [0, 16, 17, 32, 38, 41, 44, 62, 68, 71, 95]}
1210,260  :  {'frequency': 0.18, 'baskets': [0, 14, 16, 20, 29, 38, 41, 43, 44, 61, 62, 67, 68, 71, 76, 78, 94, 95]}
1210,1196  :  {'frequency': 0.15, 'baskets': [0, 14, 16, 20, 29, 38, 44, 61, 62, 67, 68, 71, 76, 78, 94]}
1210,1198  :  {'frequency': 0.1, 'baskets': [0, 16, 29, 61, 62, 67, 71, 76, 94, 95]}
2571,50  :  {'frequency': 0.12, 'baskets': [0, 16, 17, 32, 38, 41, 44, 56, 68, 71, 79, 90]}
2571,260  :  {'frequen

40

In [4]:
hc = HashCountersOfPairs(movies_df, UserBaskets)

User:  1 / 100
User:  2 / 100
User:  3 / 100
User:  4 / 100
User:  5 / 100
User:  6 / 100
User:  7 / 100
User:  8 / 100
User:  9 / 100
User:  10 / 100
User:  11 / 100
User:  12 / 100
User:  13 / 100
User:  14 / 100
User:  15 / 100
User:  16 / 100
User:  17 / 100
User:  18 / 100
User:  19 / 100
User:  20 / 100
User:  21 / 100
User:  22 / 100
User:  23 / 100
User:  24 / 100
User:  25 / 100
User:  26 / 100
User:  27 / 100
User:  28 / 100
User:  29 / 100
User:  30 / 100
User:  31 / 100
User:  32 / 100
User:  33 / 100
User:  34 / 100
User:  35 / 100
User:  36 / 100
User:  37 / 100
User:  38 / 100
User:  39 / 100
User:  40 / 100
User:  41 / 100
User:  42 / 100
User:  43 / 100
User:  44 / 100
User:  45 / 100
User:  46 / 100
User:  47 / 100
User:  48 / 100
User:  49 / 100
User:  50 / 100
User:  51 / 100
User:  52 / 100
User:  53 / 100
User:  54 / 100
User:  55 / 100
User:  56 / 100
User:  57 / 100
User:  58 / 100
User:  59 / 100
User:  60 / 100
User:  61 / 100
User:  62 / 100
User:  63 / 100
U

In [14]:
count = 0
for i in hc.keys():
    if hc[i] >= 10:
        print(i, " : ", hc[i])
        count += 1

count

260,50  :  13
527,50  :  10
1196,260  :  19
1198,260  :  15
1198,1196  :  11
1210,50  :  11
1210,260  :  18
1210,1196  :  15
1210,1198  :  10
2571,50  :  12
2571,260  :  16
2571,1196  :  12
2571,1198  :  13
2571,1210  :  16
2959,2571  :  10
593,260  :  13
356,110  :  10
356,318  :  12
593,50  :  13
593,356  :  12
4993,260  :  10
5952,4993  :  12
318,50  :  12
527,318  :  10
2028,356  :  10
296,50  :  12
296,260  :  11
318,296  :  11
593,296  :  11
593,527  :  10
858,296  :  10
1210,296  :  10
2571,296  :  11
2571,318  :  11
2571,593  :  12
2571,2028  :  12
4993,2571  :  11
7153,2571  :  10
7153,4993  :  12
79132,58559  :  10


40

In [22]:
tcl, tm = TriangularMatrixOfPairsCounters(movies_df, UserBaskets)

User:  1 / 100
User:  2 / 100
User:  3 / 100
User:  4 / 100
User:  5 / 100
User:  6 / 100
User:  7 / 100
User:  8 / 100
User:  9 / 100
User:  10 / 100
User:  11 / 100
User:  12 / 100
User:  13 / 100
User:  14 / 100
User:  15 / 100
User:  16 / 100
User:  17 / 100
User:  18 / 100
User:  19 / 100
User:  20 / 100
User:  21 / 100
User:  22 / 100
User:  23 / 100
User:  24 / 100
User:  25 / 100
User:  26 / 100
User:  27 / 100
User:  28 / 100
User:  29 / 100
User:  30 / 100
User:  31 / 100
User:  32 / 100
User:  33 / 100
User:  34 / 100
User:  35 / 100
User:  36 / 100
User:  37 / 100
User:  38 / 100
User:  39 / 100
User:  40 / 100
User:  41 / 100
User:  42 / 100
User:  43 / 100
User:  44 / 100
User:  45 / 100
User:  46 / 100
User:  47 / 100
User:  48 / 100
User:  49 / 100
User:  50 / 100
User:  51 / 100
User:  52 / 100
User:  53 / 100
User:  54 / 100
User:  55 / 100
User:  56 / 100
User:  57 / 100
User:  58 / 100
User:  59 / 100
User:  60 / 100
User:  61 / 100
User:  62 / 100
User:  63 / 100
U

In [27]:
count = 0

for i, f in enumerate(tcl):
    if f >= 10:
        x,y = tm[i]
        count += 1
        print(movies_df.iloc[x]["movieId"], movies_df.iloc[x]["movieId"])
        
        break

91


26